<a href="https://colab.research.google.com/github/ImNotDanish05/2025_KC_TI-1A/blob/main/P11_Convolutional%20Neural%20Network%20(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import tensorflow as tf
import os
print(tf.__version__)

2.18.0


In [9]:
"""
List Dir: https://api.github.com/repos/ImNotDanish05/2025_KC_TI-1A/contents/Data/P11/images
Open File: https://raw.githubusercontent.com/ImNotDanish05/2025_KC_TI-1A/main/Data/P11/images
"""

base_dir = 'https://api.github.com/repos/ImNotDanish05/2025_KC_TI-1A/contents/Data/P11/images'
train_dir = base_dir + '/train'
validation_dir = base_dir + '/val'

In [50]:
import requests
from urllib.parse import urlparse
def listdirx(dirx):
  response = requests.get(dirx)
  if response.status_code == 200:
      files = response.json()
      print(dirx)
      for file in files:
          print(f"- {file['name']}")
  else:
      print(f"Gagal mengambil data. Status: {response.status_code}")
def list_all_png_from_github_api(api_url, collected=[]):
    response = requests.get(api_url)
    if response.status_code != 200:
        print(f"❌ Gagal mengakses {api_url} | Status: {response.status_code}")
        return collected

    items = response.json()
    for item in items:
        if item['type'] == 'file' and item['name'].endswith('.png'):
            collected.append(item['download_url'])
        elif item['type'] == 'dir':
            list_all_png_from_github_api(item['url'], collected)
    return collected
def download_images(image_urls, save_dir):
    os.makedirs(save_dir, exist_ok=True)
    for url in image_urls:
        filename = os.path.basename(urlparse(url).path)
        save_path = os.path.join(save_dir, filename)

        response = requests.get(url)
        if response.status_code == 200:
            with open(save_path, 'wb') as f:
                f.write(response.content)
            print(f"✅ Downloaded: {filename}")
        else:
            print(f"❌ Gagal download: {filename}")

In [24]:
listdirx(train_dir)
listdirx(validation_dir)

https://api.github.com/repos/ImNotDanish05/2025_KC_TI-1A/contents/Data/P11/images/train
- .DS_Store
- clean
- messy
https://api.github.com/repos/ImNotDanish05/2025_KC_TI-1A/contents/Data/P11/images/val
- clean
- messy


TensorFlow

Documentation: https://keras.io/api/data_loading/image/

In [ ]:
train_clean_urls = list_all_png_from_github_api(train_dir + "/clean")
train_messy_urls = list_all_png_from_github_api(train_dir + "/messy")
validation_clean_urls = list_all_png_from_github_api(validation_dir + "/clean")
validation_messy_urls = list_all_png_from_github_api(validation_dir + "/messy")
download_images(train_clean_urls, 'dataset/train/clean')
download_images(train_messy_urls, 'dataset/train/messy')
download_images(validation_clean_urls, 'dataset/val/clean')
download_images(validation_messy_urls, 'dataset/val/messy')

In [69]:
print("Clean:", len(os.listdir('/content/dataset/train/clean')))
print("Messy:", len(os.listdir('/content/dataset/train/messy')))
print("Clean:", len(os.listdir('/content/dataset/val/clean')))
print("Messy:", len(os.listdir('/content/dataset/val/messy')))

Clean: 96
Messy: 96
Clean: 96
Messy: 96


In [83]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

test_datagen = ImageDataGenerator(
                    rescale=1./255)

In [84]:
train_generator = train_datagen.flow_from_directory(
        "/content/dataset/train",  # direktori data latih
        target_size=(150, 150),  # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4,
        # karena ini merupakan masalah klasifikasi 2 kelas, gunakan class_mode = 'binary'
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        "/content/dataset/val", # direktori data validasi
        target_size=(150, 150), # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4, # karena ini merupakan masalah klasifikasi 2 kelas gunakan class_mode = 'binary'
        class_mode='binary')
print(train_generator.class_indices)

Found 192 images belonging to 2 classes.
Found 192 images belonging to 2 classes.
{'clean': 0, 'messy': 1}


In [85]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [86]:
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_32 (Conv2D)              │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_32 (MaxPooling2D) │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_33 (Conv2D)              │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_33 (MaxPooling2D) │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_34 (Conv2D)              │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_34 (MaxPooling2D) │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_35 (Conv2D)              │ (None, 15, 15, 512)    │       590,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_35 (MaxPooling2D) │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_8 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,529,665 (51.61 MB)

 Trainable params: 13,529,665 (51.61 MB)

 Non-trainable params: 0 (0.00 B)

In [87]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

In [89]:
steps_per_epoch = len(train_generator)
validation_steps = len(validation_generator)
model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=60,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    verbose=2
)

Epoch 1/60


KeyboardInterrupt: 

In [82]:
# Buka base model-nya, tapi jangan semua layer
base_model.trainable = True

# Bisa juga fine-tune sebagian layer:
# for layer in base_model.layers[:100]:
#     layer.trainable = False

# Re-compile model setelah unfreeze
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train lagi!
model.fit(
    train_generator,
    steps_per_epoch=25,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=5,
    verbose=2
)


Epoch 1/10
25/25 - 54s - 2s/step - accuracy: 0.4300 - loss: 0.8118 - val_accuracy: 0.6000 - val_loss: 0.6411
Epoch 2/10
25/25 - 10s - 409ms/step - accuracy: 0.5000 - loss: 0.7258 - val_accuracy: 0.5000 - val_loss: 0.6362
Epoch 3/10
25/25 - 41s - 2s/step - accuracy: 0.4800 - loss: 0.7613 - val_accuracy: 0.3000 - val_loss: 0.8172
Epoch 4/10
25/25 - 10s - 415ms/step - accuracy: 0.4674 - loss: 0.7452 - val_accuracy: 0.6000 - val_loss: 0.6360
Epoch 5/10
25/25 - 17s - 674ms/step - accuracy: 0.6100 - loss: 0.7061 - val_accuracy: 0.3000 - val_loss: 0.8684
Epoch 6/10
25/25 - 10s - 403ms/step - accuracy: 0.3804 - loss: 0.7746 - val_accuracy: 0.3500 - val_loss: 0.7501
Epoch 7/10
25/25 - 12s - 462ms/step - accuracy: 0.5400 - loss: 0.7488 - val_accuracy: 0.4000 - val_loss: 0.7481
Epoch 8/10
25/25 - 11s - 443ms/step - accuracy: 0.4674 - loss: 0.7392 - val_accuracy: 0.6000 - val_loss: 0.6744
Epoch 9/10
25/25 - 10s - 408ms/step - accuracy: 0.5100 - loss: 0.7412 - val_accuracy: 0.5500 - val_loss: 0.735

# **Testing**

In [39]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

test_image = "https://i.pinimg.com/736x/31/af/bb/31afbb9580927635c0a076f99bb696fb.jpg"
download_images([test_image], 'dataset/test')

def predict_image_class(model, image_path):
    img = Image.open(image_path).resize((150, 150))
    imgplot = plt.imshow(img)
    x = np.expand_dims(np.array(img), axis=0)
    classes = model.predict(x, batch_size=10)

    if classes[0] == 0:
        print('clean')
    else:
        print('messy')

# Misalkan model adalah objek model yang sudah diinisialisasi sebelumnya

# Mengunggah gambar
image_path = "/content/dataset/test/31afbb9580927635c0a076f99bb696fb.jpg"
predict_image_class(model, image_path)

✅ Downloaded: 31afbb9580927635c0a076f99bb696fb.jpg
